# 2022 / 12 / 24

In [1]:
using IntervalArithmetic
using LaTeXStrings

In [2]:
include("./concatImportanceMethod/concatImportanceMethod.jl")
include("./concatMatricesMethod/concatMatricesMethod.jl")
include("./display/concatImportanceMethod.jl")
include("./display/concatMatricesMethod.jl")
include("./display/interval.jl")
include("./display/twofoldInterval.jl")
include("./twofoldIntervalPCM/twofoldIntervalPCM.jl")

intervalPCM2TwofoldIntervalPCM

In [3]:
function concatMatricesMethod(A₁::Matrix{Interval{T}}, A₂::Matrix{Interval{T}}) where {T <: Real}
    A = intervalPCM2TwofoldIntervalPCM(A₁, A₂)
    display(L"\text{修正前の行列}")
    display(L"A = %$(twofoldIntervalMatrixLaTeXString(A))")

    result = solveConcatMatricesMethodLP(A)
    display(L"\text{LPの最適解}")
    displayConcatMatricesMethodLPResult(result)

    Â = updateConcatMatricesMethodPCM(A, result)
    display(L"\text{修正後の行列}")
    display(L"\hat{A} = %$(twofoldIntervalMatrixLaTeXString(Â))")

    return Â
end

concatMatricesMethod (generic function with 1 method)

In [4]:
function concatImportanceMethod(A₁::Matrix{Interval{T}}, A₂::Matrix{Interval{T}}) where {T <: Real}
    approximationResults = solveConcatImportanceMethodApproximationLP.([A₁, A₂])
    display(L"\text{各DMの上近似と下近似}")
    displayConcatImportanceMethodApproximationLPResults(approximationResults)

    tBoundaries = calculateConcatImportanceMethodTBoundaries.(approximationResults)
    display(L"\text{各DMの上近似と下近似の}t\text{の範囲}")
    displayConcatImportanceMethodTBoundaries(tBoundaries)

    result = solveConcatImportanceMethodConcatLP(approximationResults, tBoundaries)
    display(L"\text{LPの最適解}")
    displayConcatImportanceMethodConcatLPResult(result)

    Â = generateConcatImportanceMethodPCM(result)
    display(L"\text{結合後のPCM}")
    display(L"\hat{A} = %$(twofoldIntervalMatrixLaTeXString(Â))")

    return Â
end

concatImportanceMethod (generic function with 1 method)

In [5]:
function runBothMethod(A₁::Matrix{Interval{T}}, A₂::Matrix{Interval{T}}) where {T <: Real}
    display(L"A_1 = %$(intervalMatrixLaTeXString(A₁))")
    display(L"A_2 = %$(intervalMatrixLaTeXString(A₂))")
    
    display(L"\text{--- PCMを結合 ---}")
    concatMatricesMethod(A₁, A₂)

    display(L"\text{--- 重要度を結合 ---}")
    concatImportanceMethod(A₁, A₂)
end

runBothMethod (generic function with 1 method)

---
推移率が成り立つcrisp PCM
$
\begin{bmatrix} 1 & 4 & 2 \\ 1/4 & 1 & 1/2 \\ 1/2 & 2 & 1 \end{bmatrix}
$
2つを入力

In [6]:
A₁ = [
    1..1 4..4 2..2
    (1/4)..(1/4) 1..1 (1/2)..(1/2)
    (1/2)..(1/2) 2..2 1..1
]
A₂ = [
    1..1 4..4 2..2
    (1/4)..(1/4) 1..1 (1/2)..(1/2)
    (1/2)..(1/2) 2..2 1..1
]

runBothMethod(A₁, A₂)
print()

L"$A_1 = \begin{bmatrix} \left[ 1.0, 1.0 \right] & \left[ 4.0, 4.0 \right] & \left[ 2.0, 2.0 \right] \\ \left[ 0.25, 0.25 \right] & \left[ 1.0, 1.0 \right] & \left[ 0.5, 0.5 \right] \\ \left[ 0.5, 0.5 \right] & \left[ 2.0, 2.0 \right] & \left[ 1.0, 1.0 \right] \end{bmatrix}$"

L"$A_2 = \begin{bmatrix} \left[ 1.0, 1.0 \right] & \left[ 4.0, 4.0 \right] & \left[ 2.0, 2.0 \right] \\ \left[ 0.25, 0.25 \right] & \left[ 1.0, 1.0 \right] & \left[ 0.5, 0.5 \right] \\ \left[ 0.5, 0.5 \right] & \left[ 2.0, 2.0 \right] & \left[ 1.0, 1.0 \right] \end{bmatrix}$"

L"$\text{--- PCMを結合 ---}$"

L"$\text{修正前の行列}$"

"\$A = \\begin{bmatrix} \\left[   1.0, \\left[   1.0,   1.0 \\right],   1.0 \\right] & \\left[   4.0, \\left[   4.0,   4.0 \\right],   4.0 \\right] & \\left[   2.0, \\left[   2.0,   2.0 \\right],   2.0 \\right] \\\\ \\left[  0.25, \\left[  0.25,  0.25 \\right],  0.25 \\right] & \\left[   1.0," ⋯ 24 bytes ⋯ "ght],   1.0 \\right] & \\left[   0.5, \\left[   0.5,   0.5 \\right],   0.5 \\right] \\\\ \\left[   0.5, \\left[   0.5,   0.5 \\right],   0.5 \\right] & \\left[   2.0, \\left[   2.0,   2.0 \\right],   2.0 \\right] & \\left[   1.0, \\left[   1.0,   1.0 \\right],   1.0 \\right] \\end{bmatrix}\$"

...

L"$W = \begin{bmatrix}\left[ 0.571, 0.571 \right] \\ \left[ 0.143, 0.143 \right] \\ \left[ 0.286, 0.286 \right]\end{bmatrix}$"

L"$v^{\text{L}-} = \begin{bmatrix} 0.571  \\  0.143  \\  0.286 \end{bmatrix}, ~~
v^{\text{U}-} = \begin{bmatrix} 0.571  \\  0.143  \\  0.286 \end{bmatrix}, ~~
v^{\text{L}+} = \begin{bmatrix} 0.571  \\  0.143  \\  0.286 \end{bmatrix}, ~~
v^{\text{U}+} = \begin{bmatrix} 0.571  \\  0.143  \\  0.286 \end{bmatrix}
$"

L"$\varepsilon^\text{L} = \begin{bmatrix} 0.0  \\  0.0  \\  0.0 \end{bmatrix}, ~~
\varepsilon^\text{U} = \begin{bmatrix} 0.0  \\  0.0  \\  0.0 \end{bmatrix}
$"

L"$\text{結合後のPCM}$"

"\$\\hat{A} = \\begin{bmatrix} \\left[   1.0, \\left[   1.0,   1.0 \\right],   1.0 \\right] & \\left[   4.0, \\left[   4.0,   4.0 \\right],   4.0 \\right] & \\left[   2.0, \\left[   2.0,   2.0 \\right],   2.0 \\right] \\\\ \\left[  0.25, \\left[  0.25,  0.25 \\right],  0.25 \\right] & \\left[ " ⋯ 30 bytes ⋯ "ght],   1.0 \\right] & \\left[   0.5, \\left[   0.5,   0.5 \\right],   0.5 \\right] \\\\ \\left[   0.5, \\left[   0.5,   0.5 \\right],   0.5 \\right] & \\left[   2.0, \\left[   2.0,   2.0 \\right],   2.0 \\right] & \\left[   1.0, \\left[   1.0,   1.0 \\right],   1.0 \\right] \\end{bmatrix}\$"

---
$
\begin{bmatrix} 1 & 3 & 1/3 \\ 1/3 & 1 & 1/10 \\ 3 & 10 & 1 \end{bmatrix}
$
に近いPCM2つを入力

In [7]:
A₁ = [
    1..1 2.7..3.2 (1/3.3)..(1/3)
    (1/3.2)..(1/2.7) 1..1 (1/10)..(1/9.7)
    3..3.3 9.7..10 1..1
]
A₂ = [
    1..1 2.2..2.8 (1/4)..(1/2.8)
    (1/2.8)..(1/2.2) 1..1 (1/9.8)..(1/9.7)
    2.8..4 9.7..9.8 1..1
]

runBothMethod(A₁, A₂)
print()

L"$A_1 = \begin{bmatrix} \left[ 1.0, 1.0 \right] & \left[ 2.7, 3.2 \right] & \left[ 0.303, 0.333 \right] \\ \left[ 0.312, 0.37 \right] & \left[ 1.0, 1.0 \right] & \left[ 0.1, 0.103 \right] \\ \left[ 3.0, 3.3 \right] & \left[ 9.7, 10.0 \right] & \left[ 1.0, 1.0 \right] \end{bmatrix}$"

L"$A_2 = \begin{bmatrix} \left[ 1.0, 1.0 \right] & \left[ 2.2, 2.8 \right] & \left[ 0.25, 0.357 \right] \\ \left[ 0.357, 0.455 \right] & \left[ 1.0, 1.0 \right] & \left[ 0.102, 0.103 \right] \\ \left[ 2.8, 4.0 \right] & \left[ 9.7, 9.8 \right] & \left[ 1.0, 1.0 \right] \end{bmatrix}$"

L"$\text{--- PCMを結合 ---}$"

L"$\text{修正前の行列}$"

"\$A = \\begin{bmatrix} \\left[   1.0, \\left[   1.0,   1.0 \\right],   1.0 \\right] & \\left[   2.2, \\left[   2.7,   2.8 \\right],   3.2 \\right] & \\left[  0.25, \\left[ 0.303, 0.333 \\right], 0.357 \\right] \\\\ \\left[ 0.312, \\left[ 0.357,  0.37 \\right], 0.455 \\right] & \\left[   1.0," ⋯ 24 bytes ⋯ "ght],   1.0 \\right] & \\left[   0.1, \\left[ 0.102, 0.103 \\right], 0.103 \\right] \\\\ \\left[   2.8, \\left[   3.0,   3.3 \\right],   4.0 \\right] & \\left[   9.7, \\left[   9.7,   9.8 \\right],  10.0 \\right] & \\left[   1.0, \\left[   1.0,   1.0 \\right],   1.0 \\right] \\end{bmatrix}\$"

...

L"$W = \begin{bmatrix}\left[ 0.214, 0.23 \right] \\ \left[ 0.073, 0.079 \right] \\ \left[ 0.697, 0.707 \right]\end{bmatrix}$"

L"$v^{\text{L}-} = \begin{bmatrix} 0.199  \\  0.072  \\  0.698 \end{bmatrix}, ~~
v^{\text{U}-} = \begin{bmatrix} 0.23  \\  0.075  \\  0.726 \end{bmatrix}, ~~
v^{\text{L}+} = \begin{bmatrix} 0.214  \\  0.075  \\  0.697 \end{bmatrix}, ~~
v^{\text{U}+} = \begin{bmatrix} 0.214  \\  0.075  \\  0.697 \end{bmatrix}
$"

L"$\varepsilon^\text{L} = \begin{bmatrix} 0.015  \\  0.003  \\  0.0 \end{bmatrix}, ~~
\varepsilon^\text{U} = \begin{bmatrix} 0.0  \\  0.0  \\  0.019 \end{bmatrix}
$"

L"$\text{結合後のPCM}$"

"\$\\hat{A} = \\begin{bmatrix} \\left[   1.0, \\left[   1.0,   1.0 \\right],   1.0 \\right] & \\left[ 2.513, \\left[ 2.861, 3.077 \\right],   3.2 \\right] & \\left[ 0.282, \\left[ 0.303,  0.33 \\right],  0.33 \\right] \\\\ \\left[ 0.312, \\left[ 0.325, 0.349 \\right], 0.398 \\right] & \\left[ " ⋯ 30 bytes ⋯ "ght],   1.0 \\right] & \\left[ 0.102, \\left[ 0.106, 0.107 \\right], 0.114 \\right] \\\\ \\left[ 3.028, \\left[ 3.031,   3.3 \\right], 3.545 \\right] & \\left[ 8.789, \\left[ 9.315, 9.443 \\right], 9.842 \\right] & \\left[   1.0, \\left[   1.0,   1.0 \\right],   1.0 \\right] \\end{bmatrix}\$"

---

In [8]:
A₁ = [
    1..1 1..3 1..5
    (1/3)..1 1..1 1..9
    (1/5)..1 (1/9)..1 1..1
]
A₂ = [
    1..1 1..1 1..2
    1..1 1..1 (1/2)..1
    (1/2)..1 1..2 1..1
]

runBothMethod(A₁, A₂)
print()

L"$A_1 = \begin{bmatrix} \left[ 1.0, 1.0 \right] & \left[ 1.0, 3.0 \right] & \left[ 1.0, 5.0 \right] \\ \left[ 0.333, 1.0 \right] & \left[ 1.0, 1.0 \right] & \left[ 1.0, 9.0 \right] \\ \left[ 0.2, 1.0 \right] & \left[ 0.111, 1.0 \right] & \left[ 1.0, 1.0 \right] \end{bmatrix}$"

L"$A_2 = \begin{bmatrix} \left[ 1.0, 1.0 \right] & \left[ 1.0, 1.0 \right] & \left[ 1.0, 2.0 \right] \\ \left[ 1.0, 1.0 \right] & \left[ 1.0, 1.0 \right] & \left[ 0.5, 1.0 \right] \\ \left[ 0.5, 1.0 \right] & \left[ 1.0, 2.0 \right] & \left[ 1.0, 1.0 \right] \end{bmatrix}$"

L"$\text{--- PCMを結合 ---}$"

L"$\text{修正前の行列}$"

"\$A = \\begin{bmatrix} \\left[   1.0, \\left[   1.0,   1.0 \\right],   1.0 \\right] & \\left[   1.0, \\left[   1.0,   1.0 \\right],   3.0 \\right] & \\left[   1.0, \\left[   1.0,   2.0 \\right],   5.0 \\right] \\\\ \\left[ 0.333, \\left[   1.0,   1.0 \\right],   1.0 \\right] & \\left[   1.0," ⋯ 24 bytes ⋯ "ght],   1.0 \\right] & \\left[   0.5, \\left[   1.0,   1.0 \\right],   9.0 \\right] \\\\ \\left[   0.2, \\left[   0.5,   1.0 \\right],   1.0 \\right] & \\left[ 0.111, \\left[   1.0,   1.0 \\right],   2.0 \\right] & \\left[   1.0, \\left[   1.0,   1.0 \\right],   1.0 \\right] \\end{bmatrix}\$"

...

L"$W = \begin{bmatrix}\left[ 0.333, 0.545 \right] \\ \left[ 0.333, 0.345 \right] \\ \left[ 0.109, 0.333 \right]\end{bmatrix}$"

L"$v^{\text{L}-} = \begin{bmatrix} 0.333  \\  0.333  \\  0.109 \end{bmatrix}, ~~
v^{\text{U}-} = \begin{bmatrix} 0.545  \\  0.345  \\  0.333 \end{bmatrix}, ~~
v^{\text{L}+} = \begin{bmatrix} 0.409  \\  0.222  \\  0.222 \end{bmatrix}, ~~
v^{\text{U}+} = \begin{bmatrix} 0.409  \\  0.222  \\  0.222 \end{bmatrix}
$"

L"$\varepsilon^\text{L} = \begin{bmatrix} 0.076  \\  0.0  \\  0.113 \end{bmatrix}, ~~
\varepsilon^\text{U} = \begin{bmatrix} 0.101  \\  0.012  \\  0.0 \end{bmatrix}
$"

L"$\text{結合後のPCM}$"

"\$\\hat{A} = \\begin{bmatrix} \\left[   1.0, \\left[   1.0,   1.0 \\right],   1.0 \\right] & \\left[ 0.965, \\left[ 1.184, 1.333 \\right], 2.455 \\right] & \\left[   1.0, \\left[ 1.227, 2.455 \\right],   5.0 \\right] \\\\ \\left[ 0.407, \\left[  0.75, 0.844 \\right], 1.036 \\right] & \\left[ " ⋯ 30 bytes ⋯ "ght],   1.0 \\right] & \\left[ 0.667, \\left[   1.0, 1.555 \\right], 3.167 \\right] \\\\ \\left[   0.2, \\left[ 0.407, 0.815 \\right],   1.0 \\right] & \\left[ 0.316, \\left[ 0.643,   1.0 \\right],   1.5 \\right] & \\left[   1.0, \\left[   1.0,   1.0 \\right],   1.0 \\right] \\end{bmatrix}\$"

---
次の区間行列 $A_1, A_2$ は上・下近似を求めるLPで実行不可能になる

In [9]:
A₁ = [
    1..1 (1/5.5)..(1/4.8) 2.7..3.3 1.5..2.1
    4.8..5.5 1..1 6.8..7.6 0.8..1.1
    (1/3.3)..(1/2.7) (1/7.6)..(1/6.8) 1..1 (1/3.4)..(1/3)
    (1/2.1)..(1/1.5) (1/1.1)..(1/0.8) 3..3.4 1..1
]
A₂ = [
    1..1 (1/5.5)..(1/5.1) 2.7..2.9 1.5..2
    5.1..5.5 1..1 6.9..7.2 0.9..1.3
    (1/2.9)..(1/2.7) (1/7.2)..(1/6.9) 1..1 (1/3.4)..(1/3)
    (1/2)..(1/1.5) (1/1.3)..(1/0.9) 3..3.4 1..1
]

runBothMethod(A₁, A₂)
print()

L"$A_1 = \begin{bmatrix} \left[ 1.0, 1.0 \right] & \left[ 0.182, 0.208 \right] & \left[ 2.7, 3.3 \right] & \left[ 1.5, 2.1 \right] \\ \left[ 4.8, 5.5 \right] & \left[ 1.0, 1.0 \right] & \left[ 6.8, 7.6 \right] & \left[ 0.8, 1.1 \right] \\ \left[ 0.303, 0.37 \right] & \left[ 0.132, 0.147 \right] & \left[ 1.0, 1.0 \right] & \left[ 0.294, 0.333 \right] \\ \left[ 0.476, 0.667 \right] & \left[ 0.909, 1.25 \right] & \left[ 3.0, 3.4 \right] & \left[ 1.0, 1.0 \right] \end{bmatrix}$"

L"$A_2 = \begin{bmatrix} \left[ 1.0, 1.0 \right] & \left[ 0.182, 0.196 \right] & \left[ 2.7, 2.9 \right] & \left[ 1.5, 2.0 \right] \\ \left[ 5.1, 5.5 \right] & \left[ 1.0, 1.0 \right] & \left[ 6.9, 7.2 \right] & \left[ 0.9, 1.3 \right] \\ \left[ 0.345, 0.37 \right] & \left[ 0.139, 0.145 \right] & \left[ 1.0, 1.0 \right] & \left[ 0.294, 0.333 \right] \\ \left[ 0.5, 0.667 \right] & \left[ 0.769, 1.111 \right] & \left[ 3.0, 3.4 \right] & \left[ 1.0, 1.0 \right] \end{bmatrix}$"

L"$\text{--- PCMを結合 ---}$"

L"$\text{修正前の行列}$"

"\$A = \\begin{bmatrix} \\left[   1.0, \\left[   1.0,   1.0 \\right],   1.0 \\right] & \\left[ 0.182, \\left[ 0.182, 0.196 \\right], 0.208 \\right] & \\left[   2.7, \\left[   2.7,   2.9 \\right],   3.3 \\right] & \\left[   1.5, \\left[   1.5,   2.0 \\right],   2.1 \\right] \\\\ \\left[   4.8," ⋯ 439 bytes ⋯ "ht], 0.333 \\right] \\\\ \\left[ 0.476, \\left[   0.5, 0.667 \\right], 0.667 \\right] & \\left[ 0.769, \\left[ 0.909, 1.111 \\right],  1.25 \\right] & \\left[   3.0, \\left[   3.0,   3.4 \\right],   3.4 \\right] & \\left[   1.0, \\left[   1.0,   1.0 \\right],   1.0 \\right] \\end{bmatrix}\$"

...

L"$W = \begin{bmatrix}\left[ 0.075, 0.077 \right] \\ \left[ 0.41, 0.41 \right] \\ \left[ 0.057, 0.059 \right] \\ \left[ 0.456, 0.456 \right]\end{bmatrix}$"

L"$v^{\text{L}-} = \begin{bmatrix} 0.075  \\  0.41  \\  0.057  \\  0.187 \end{bmatrix}, ~~
v^{\text{U}-} = \begin{bmatrix} 0.0  \\  0.0  \\  0.0  \\  0.0 \end{bmatrix}, ~~
v^{\text{L}+} = \begin{bmatrix} 0.075  \\  0.41  \\  0.057  \\  0.187 \end{bmatrix}, ~~
v^{\text{U}+} = \begin{bmatrix} 0.075  \\  0.41  \\  0.057  \\  0.187 \end{bmatrix}
$"

L"$\varepsilon^\text{L} = \begin{bmatrix} 0.0  \\  0.0  \\  0.0  \\  0.0 \end{bmatrix}, ~~
\varepsilon^\text{U} = \begin{bmatrix} 0.0  \\  0.0  \\  0.0  \\  0.0 \end{bmatrix}
$"

L"$\text{結合後のPCM}$"

"\$\\hat{A} = \\begin{bmatrix} \\left[   1.0, \\left[   1.0,   1.0 \\right],   1.0 \\right] & \\left[ 0.182, \\emptyset, 0.188 \\right] & \\left[ 1.255, \\emptyset, 1.353 \\right] & \\left[ 0.164, \\emptyset, 0.413 \\right] \\\\ \\left[ 5.323, \\emptyset,   5.5 \\right] & \\left[   1.0, \\left[" ⋯ 229 bytes ⋯ "0,   1.0 \\right],   1.0 \\right] & \\left[ 0.125, \\emptyset, 0.318 \\right] \\\\ \\left[ 2.424, \\emptyset, 6.111 \\right] & \\left[ 0.455, \\emptyset, 1.111 \\right] & \\left[ 3.142, \\emptyset,   8.0 \\right] & \\left[   1.0, \\left[   1.0,   1.0 \\right],   1.0 \\right] \\end{bmatrix}\$"